In [ ]:
!pip install huggingface_hub
!pip install --upgrade transformers
!pip install -U "huggingface_hub[cli]"

In [3]:
# Standard Library Imports
import warnings

# PyTorch
import torch

## Hugging Face Transformers
from transformers import pipeline
from transformers.utils import logging

# Hugging Face Hub
from huggingface_hub import login

## Google Colab
from google.colab import userdata

# Configuration
logging.set_verbosity_error()
warnings.filterwarnings('ignore')

In [4]:
# Login using the API key
login(userdata.get("HUGGINGFACEHUB_API"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
if device != "cuda":
    print("GPU not available")
else:
    print("GPU available\n\n")
    print()

GPU available





In [ ]:
# Building a conversation pipeline
ChatBot = pipeline("text2text-generation", model="facebook/blenderbot-400M-distill", device=device)

In [7]:
user_message = """
What are some fun activities I can do in the winter?
"""

In [8]:
conversation = ChatBot(user_message)
conversation[0]['generated_text']

' I like snowboarding and skiing.  What do you like to do in winter?'

### Second method

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

In [10]:
# define list to capture message history
conversation_history = []
history_string = '\n'.join(conversation_history)

In [11]:
# tokenize the user message
user_input = tokenizer(history_string, user_message, return_tensors="pt")
user_input

{'input_ids': tensor([[ 228,  206, 6524,  366,  459,  941, 3275, 1380,  281,  396,  361,  302,
          271, 5602,   38,  206]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [12]:
# model output
output = model.generate(**user_input)
output

tensor([[   1,    1,  281,  913, 5602, 3275, 1380,  371,  731,   21,  281,  398,
          287,  425, 1025,   80,  278,  298, 4250, 2962,  278,   21,    2]])

In [13]:
# decode the message
model_response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
model_response

'I love winter activities as well. I like to go skiing and snowboarding.'

In [14]:
# keep the history
conversation_history.append(user_message)
conversation_history.append(model_response)

In [15]:
history_string='\n'.join(conversation_history)
history_string

'\nWhat are some fun activities I can do in the winter?\n\nI love winter activities as well. I like to go skiing and snowboarding.'

# Looping over the code

In [16]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

# Initialize conversation history
conversation_history = []

# Explicitly name the function for user input to avoid any conflict
def get_user_input():
    return input("User: ")

while True:
    # Get user input
    user_message = get_user_input()

    # Exit condition
    if user_message.lower() in ["exit", "quit"]:
        print("Ending conversation.")
        break

    # Append user message to history
    conversation_history.append(f"User: {user_message}")

    # Prepare input for the model
    history_string = '\n'.join(conversation_history)
    tokenized_input = tokenizer(history_string, return_tensors="pt")

    # Generate model response
    output_tokens = model.generate(**tokenized_input)
    model_response = tokenizer.decode(output_tokens[0], skip_special_tokens=True).strip()

    # Print and append model response to history
    print(f"Bot: {model_response}")
    conversation_history.append(f"Bot: {model_response}")


User: Hello...
Bot: Hello! How are you doing today? I just got back from a walk with my dog.
User: what's your dogs breed?
Bot: I have a chihuahua and a pomeranian. Do you have any pets?
User: exit
Ending conversation.
